In [8]:
pip install gradio llama-index-core llama-index-readers-file pypdf groq



SyntaxError: invalid syntax (2343462114.py, line 1)

In [9]:
pip install gradio llama-index-core llama-index-readers-file pypdf groq



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import gradio as gr
from groq import Groq
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import tempfile

# Defina sua chave da API do GROQ
GROQ_API_KEY = os.getenv('GROQ_API_KEY', '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')  # Substitua pela sua chave

# Inicializa o cliente da API Groq
client = Groq(api_key=GROQ_API_KEY)

# Variável global para armazenar o índice do documento
index = None

def processar_pdf(file):
    """Lê o PDF e cria um índice para responder perguntas baseadas no conteúdo."""
    global index

    if file is None:
        return "❌ Nenhum arquivo enviado."

    # Salva temporariamente o arquivo
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(file.read())
        temp_path = temp_file.name

    # Carrega o documento
    reader = SimpleDirectoryReader(input_files=[temp_path])
    documents = reader.load_data()

    # Cria um índice para busca no PDF
    index = VectorStoreIndex.from_documents(documents)
    return "✅ PDF carregado! Agora você pode fazer perguntas."

def assistente_agricultura(user_prompt):
    """Responde perguntas sobre o conteúdo do PDF processado."""
    if index is None:
        return "❌ Nenhum PDF carregado ainda. Faça o upload antes de perguntar."

    # Cria um motor de busca no índice
    query_engine = index.as_query_engine()

    # Faz a consulta e retorna a resposta
    resposta = query_engine.query(user_prompt)
    return str(resposta)

# Configuração da interface Gradio
with gr.Blocks() as iface:
    gr.Markdown("# 🌾 Assistente Agro IA - Análise de PDF")
    gr.Markdown("📥 Faça o upload de um PDF e faça perguntas sobre seu conteúdo!")

    # Upload do arquivo PDF
    with gr.Row():
        pdf_input = gr.File(label="Envie um PDF", type="filepath")
        process_button = gr.Button("Processar PDF")

    # Campo de entrada e saída para perguntas
    with gr.Row():
        question_input = gr.Textbox(lines=2, placeholder="Digite sua pergunta...")
        submit_button = gr.Button("Perguntar")

    response_output = gr.Textbox(label="Resposta do Assistente", lines=5)

    # Conectar funções aos botões
    process_button.click(processar_pdf, inputs=[pdf_input], outputs=[response_output])
    submit_button.click(assistente_agricultura, inputs=[question_input], outputs=[response_output])

# Executa a interface web
if __name__ == "__main__":
    iface.launch()


In [2]:
import os
import gradio as gr
import fitz  # PyMuPDF para extração rápida de texto do PDF
from groq import Groq
from llama_index.core import VectorStoreIndex, Document
import tempfile

# Defina sua chave da API do GROQ
GROQ_API_KEY = os.getenv('GROQ_API_KEY', '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')  # Substitua pela sua chave

# Inicializa o cliente da API Groq
client = Groq(api_key=GROQ_API_KEY)

# Variável global para armazenar o índice do documento
index = None

def extrair_texto_pdf(file_path):
    """Extrai texto de um PDF rapidamente usando PyMuPDF."""
    doc = fitz.open(file_path)
    texto_completo = "\n".join([page.get_text("text") for page in doc])
    return texto_completo

def processar_pdf(file):
    """Processa o PDF e cria um índice para responder perguntas."""
    global index

    if file is None:
        return "❌ Nenhum arquivo enviado."

    # Salva temporariamente o arquivo
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(file.read())
        temp_path = temp_file.name

    # Extrai o texto do PDF rapidamente
    texto_extraido = extrair_texto_pdf(temp_path)

    if not texto_extraido.strip():
        return "⚠️ O PDF não contém texto legível."

    # Cria um documento e indexa para busca
    documento = Document(texto_extraido)
    index = VectorStoreIndex.from_documents([documento])

    return "✅ PDF carregado! Agora você pode fazer perguntas."

def assistente_agricultura(user_prompt):
    """Responde perguntas sobre o conteúdo do PDF processado."""
    if index is None:
        return "❌ Nenhum PDF carregado ainda. Faça o upload antes de perguntar."

    # Criar um mecanismo de busca no índice
    query_engine = index.as_query_engine()

    # Faz a consulta e retorna a resposta
    resposta = query_engine.query(user_prompt)
    return str(resposta)

# Configuração da interface Gradio
with gr.Blocks() as iface:
    gr.Markdown("# 🌾 Assistente Agro IA - Análise de PDF")
    gr.Markdown("📥 Faça o upload de um PDF e faça perguntas sobre seu conteúdo!")

    # Upload do arquivo PDF
    with gr.Row():
        pdf_input = gr.File(label="Envie um PDF", type="filepath")
        process_button = gr.Button("Processar PDF")

    # Campo de entrada e saída para perguntas
    with gr.Row():
        question_input = gr.Textbox(lines=2, placeholder="Digite sua pergunta...")
        submit_button = gr.Button("Perguntar")

    response_output = gr.Textbox(label="Resposta do Assistente", lines=5)

    # Conectar funções aos botões
    process_button.click(processar_pdf, inputs=[pdf_input], outputs=[response_output])
    submit_button.click(assistente_agricultura, inputs=[question_input], outputs=[response_output])

# Executa a interface web
if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/blocks.py", line 2098, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/gradio/blocks.py", line 1645, in call_function
    prediction = await anyio.to_thread.run_

In [8]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 3.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
